In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
!ls

env		requirements.txt	   university_data.csv
prepareData.py	universities_data_raw.csv
Projekt.ipynb	universities_ranking.json


In [3]:
# create dataframe from file
dataframe = pd.read_csv("university_data.csv")
dataframe

,Name,Ranking,Ranking display,Applicants total,Admissions total,Enrolled total,"Tuition and fees, 2010-11","Tuition and fees, 2011-12","Tuition and fees, 2012-13","Tuition and fees, 2013-14",Control of institution,Total price for in-state students living on campus 2013-14,Total price for out-of-state students living on campus 2013-14
0,Alabama A & M University,1501+,1500+,6142.0,5521.0,1104.0,5800.0,6828.0,7182.0,7182.0,Public,21849.0,27441.0
1,University of Alabama at Birmingham,1501+,1500+,5689.0,4934.0,1773.0,5806.0,6264.0,6798.0,7206.0,Public,22495.0,31687.0
2,Amridge University,1501+,1500+,NaN,NaN,NaN,8360.0,8720.0,6800.0,6870.0,Private not-for-profit,NaN,NaN
3,University of Alabama in Huntsville,1501+,1500+,2054.0,1656.0,651.0,7492.0,8094.0,8794.0,9192.0,Public,23466.0,35780.0
4,Alabama State University,1501+,1500+,10245.0,5251.0,1479.0,7164.0,8082.0,7932.0,8720.0,Public,18286.0,25222.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,University of South Florida-Sarasota-Manatee,1501+,1500+,393.0,136.0,86.0,NaN,NaN,NaN,5587.0,Public,NaN,NaN
1530,The Kingâ€™s College,1501+,1500+,3033.0,2158.0,127.0,27350.0,29240.0,29240.0,31300.0,Private not-for-profit,48717.0,48717.0
1531,Ottawa University-Online,1501+,1500+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Private not-for-profit,NaN,NaN
1532,Providence Christian College,1501+,1500+,122.0,65.0,20.0,20444.0,21444.0,22686.0,24222.0,Private not-for-profit,38602.0,38602.0


In [4]:
# Drop colums: Name, Control of Institution
dataframe = dataframe.drop("Name",axis='columns')
dataframe = dataframe.drop("Control of institution",axis='columns')
dataframe = dataframe.drop("Tuition and fees, 2010-11",axis='columns')
dataframe = dataframe.drop("Tuition and fees, 2011-12",axis='columns')
dataframe = dataframe.drop("Tuition and fees, 2012-13",axis='columns')
dataframe = dataframe.drop("Total price for in-state students living on campus 2013-14",axis='columns')

dataframe

,Ranking,Ranking display,Applicants total,Admissions total,Enrolled total,"Tuition and fees, 2013-14",Total price for out-of-state students living on campus 2013-14
0,1501+,1500+,6142.0,5521.0,1104.0,7182.0,27441.0
1,1501+,1500+,5689.0,4934.0,1773.0,7206.0,31687.0
2,1501+,1500+,NaN,NaN,NaN,6870.0,NaN
3,1501+,1500+,2054.0,1656.0,651.0,9192.0,35780.0
4,1501+,1500+,10245.0,5251.0,1479.0,8720.0,25222.0
...,...,...,...,...,...,...,...
1529,1501+,1500+,393.0,136.0,86.0,5587.0,NaN
1530,1501+,1500+,3033.0,2158.0,127.0,31300.0,48717.0
1531,1501+,1500+,NaN,NaN,NaN,NaN,NaN
1532,1501+,1500+,122.0,65.0,20.0,24222.0,38602.0


In [9]:
# Zmiana rankingu na cos bardziej "parsowalnego" bo inaczej tworzenie macierzy sie wykrzacza ze nie moze skonwertowac wartosci
for x, rank in enumerate(dataframe["Ranking"]):
    if '–' in rank:
        rank = rank.split('–')[0]
    elif '+' in rank:
        rank = rank[:len(rank)-1]
    dataframe["Ranking"].loc[x] = rank

/tmp/ipykernel_10146/3760652350.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["Ranking"].loc[x] = rank
/tmp/ipykernel_10146/3760652350.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["Ranking"].loc[x] = rank
/tmp/ipykernel_10146/3760652350.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["Ranking"].loc[x] = rank
/tmp/ipykernel_10146/3760652350.py:7: SettingWithCopyWarning: 
A value is trying t

In [10]:
# use corr() method on dataframe to
# make correlation matrix
matrix = dataframe[['Applicants total', \
                    'Ranking', \
                    'Admissions total', \
                    'Enrolled total', \
                    'Tuition and fees, 2013-14', \
                    'Total price for out-of-state students living on campus 2013-14']].corr(method='pearson')
 
# print correlation matrix
print("Correlation Matrix is : ")
print(matrix)


Correlation Matrix is : 
                                                    Applicants total  \
Applicants total                                            1.000000   
Ranking                                                    -0.490558   
Admissions total                                            0.854956   
Enrolled total                                              0.785045   
Tuition and fees, 2013-14                                  -0.031060   
Total price for out-of-state students living on...          0.275459   

                                                     Ranking  \
Applicants total                                   -0.490558   
Ranking                                             1.000000   
Admissions total                                   -0.376663   
Enrolled total                                     -0.433686   
Tuition and fees, 2013-14                          -0.113080   
Total price for out-of-state students living on... -0.263483   

                     

In [8]:
matrix.style.background_gradient(cmap='coolwarm')

,Applicants total,Ranking,Admissions total,Enrolled total,"Tuition and fees, 2013-14",Total price for out-of-state students living on campus 2013-14
Applicants total,1.000000,-0.490558,0.854956,0.785045,-0.031060,0.275459
Ranking,-0.490558,1.000000,-0.376663,-0.433686,-0.113080,-0.263483
Admissions total,0.854956,-0.376663,1.000000,0.883965,-0.176093,0.142301
Enrolled total,0.785045,-0.433686,0.883965,1.000000,-0.321142,0.008601
"Tuition and fees, 2013-14",-0.031060,-0.113080,-0.176093,-0.321142,1.000000,0.858242
Total price for out-of-state students living on campus 2013-14,0.275459,-0.263483,0.142301,0.008601,0.858242,1.000000
